# Scientific Programming in Julia

Scientific programming is a subset of generic programming with specific goals: it needs to be simple to code, understandable to read, and performant. The last requirement is particularly important as data becomes larger and more accessible and models more complex. The unfortunate reality is that typically a programming language is very readable and easy to program in (think: R and Python) or unwiedly but fast (think: C++). Julia claims to do both.

It allows us to interact with the REPL, just as we would an interpreted language, with familiar function names and a mathematical like syntax. It is also designed from the ground up to support parallel computing paradigms making it ideal for the world of high performance computing (HPC). Finally, it's compiler is a just-in-time (JIT) compiler reliant on the low level virtual machine (LLVM) and is very well optimised making it very fast - orders of magnitude faster than R or Python. In fact, most of Julia code is written in Julia itself whereas most performant Python code is written in C (this is the two-language problem). This means Julia can be used as scripting language and then quickly turned into optimised code which saves on development time.

This is quite a rosy picture - what are the drawbacks? It's quite memory hungry compared to modern alternatives such as Rust, and uses a garbage collector (this is usually not a problem for scientific programming). It's JIT compiler means that the first time to run any code is *slow* and requires us to adopt a different workflow. Its a new language: long term support (LTS) and version 1.0 came out just a few years ago. Therefore, the package ecosystem, particularly in the life sciences, is not as extensive as R's Bioconductor, for example.

These drawbacks, with the exception of packaging, are very minor. Package support is a bit of a chicken-and-egg problem but the advantages of the language make Julia very worthwhile to learn.

## 1.0 Basic Input, Data Types, and Operations

Let's go through some basic data types, how to construct them, and the notion of a type hierarchy. Assignment in Julia is done through the equals operator ``=`` (in R it might be done through ``<-``). The data types we are most familiar with are numbers and strings. Numbers can be written quite naturally and strings are encoded with double apostrophes ``""``. A useful function when querying dataypes is ``typeof``. Let's assign a few variables some data:

In [1]:
var1 = 1.0;
var2 = 3;
var3 = "three";
var4 = 1.4 + 3.0im;

We suppressed output using ``;`` but in general there is no penality for not using it. Furthermore, if we do not use it only the last line will be printed to the REPL. A useful function to get readouts to the REPL (or in this case Notebook) is the ``println`` function. Let's use the ``typeof`` function and the ``println`` function to find the types of our variables:

In [2]:
println(typeof(var1))
println(typeof(var2)) 
println(typeof(var3))
println(typeof(var4))

Float64
Int64
String
ComplexF64


In this case we have a floating point number, an integer, and a string. The 64 refers to the precision of the numbers (Strings have no precision) and this is the default precision Julia takes unless specified. It has support for 16, 32, 64 bit precision for floats, 8-128 bit precision for ints, as well as arbitrary precision through BigFloat and BigInt. To assign a variable with these precisions we can use the type constructors:

In [3]:
fl32 = Float32(1.0);
in16 = Int16(1);
bigInt = BigInt(10^1200);
bigFl = BigFloat(1.15^1290);

Using the type constructors can sometimes be a little cumbersome. Julia also provides shorthand for some common types which we can use in the form of scientific notation. These include ``f`` for ``Float32``, ``e`` for ``Float64``. Let's try these out:

In [4]:
x1 = 1.4f-5
x2 = 1.2e0
println(x1)
println(typeof(x1))
println(x2)
println(typeof(x2))

1.4e-5
Float32
1.2
Float64


### 1.2 Basic Operations

Now lets try some basic operations. The mathematical binary operators are all very intituitive and behave as expected even when multiple types are involved. Let's try a few of them out:

In [5]:
var1 + var1

2.0

In [6]:
var1 - var1

0.0

In [7]:
var2 * var1

3.0

In [8]:
var2 ^ var2

27

In [9]:
var2 / (var1 + var2)

0.75

In [10]:
var2 * var4

4.199999999999999 + 9.0im

In [11]:
var3 * var1

LoadError: MethodError: no method matching *(::String, ::Float64)
[0mClosest candidates are:
[0m  *(::Any, ::Any, [91m::Any[39m, [91m::Any...[39m) at operators.jl:591
[0m  *([91m::T[39m, ::T) where T<:Union{Float16, Float32, Float64} at float.jl:385
[0m  *(::Union{AbstractChar, AbstractString}, [91m::Union{AbstractChar, AbstractString}...[39m) at strings/basic.jl:260
[0m  ...

The last line throws an error. This is because there is no intuitive method to define the multiplication operation of a floating point number with a string. This might not be surprising: when would we ever bother with this operation? However, it does point to two important points in Julia: 1) it is intuitive and things tend to behave as you might expect, 2) functions are sensitive to input and have the possibility of taking more than one input type. The latter point is salient and we will cover it in more detail at a later stage.

What can we do with strings? The most common string operation is concatention which is also a binary operation and in Julia this is handled with ``*``. This might seem confusing at first, didn't the * operation not work on strings before? This highlights that certain functions can be overloaded to perform different functions depending on what they are operating on (this is true in R and Python, for example, ``+`` in Python concatenates strings). The ``*`` operator was chosen because string contactenation is not commutative and in Mathematics we typically use ``+`` to denote binary oeperations that are commutative. Let's try contactenating two strings:

In [12]:
str1 = "ThisMakes"
str2 = "Sense"
str_cat1 = str1 * str2
str_cat2 = str2 * str1

println(str_cat1)
println(str_cat2)

ThisMakesSense
SenseThisMakes


### 1.3 Vectors, Matrices, and Arrays

Arrays in Julia are denoted with square brackets ``[]``. They are entirely general and will asscociate the highest order type in the array. For vectors the entries are seperated by a comma. Let's construct a basic vector of floats and a vector containing some arbitrary data types:

In [13]:
v_float = [1.0, 1.4, 0.5];
v_any = ["saf", 1, 1.45, 1 + 0.4im];
println([typeof(v_float), typeof(v_any)])

DataType[Vector{Float64}, Vector{Any}]


As expected, Julia correctly assigned the data type as a vector and gave the primitive data type as Float64 for the vector with all Float64s and Any for the arbitrary vector (the highest abstraction) because it couldn't decide what should be in there. To construct a matrix we use spaces to seperate the columms and ``;`` to seperate the rows. Let's create a matrix of integers:

In [14]:
mat_int =  [1 2 3; 
            4 5 6;
            7 8 9]

3×3 Matrix{Int64}:
 1  2  3
 4  5  6
 7  8  9

We can generalise this further with additional semi-colons, but this is generally not advisable because it is unwieldly and other forms of data input are probably more efficient at this point. To demonstrate:

In [15]:
tens_int = [1 2 3; 
            4 5 6;
            7 8 9;;;
                    "a" "b" "c";
                    "d" "e" "f";
                    "h" "i" "j"]

3×3×2 Array{Any, 3}:
[:, :, 1] =
 1  2  3
 4  5  6
 7  8  9

[:, :, 2] =
 "a"  "b"  "c"
 "d"  "e"  "f"
 "h"  "i"  "j"

### 1.3.1 Indexing
Suppose that we want to retrieve data from an assigned variable; we need to index into it. Indexing is perfomed with square brackets following a variable name and can be done in several ways: ``var[dim1][dim2][dim3]...`` or ``var[dim1, dim2, dim3...]``, or ``var[ind]``. The last method steps through the first slice of dim1 then dim2 then dim3 i.e. through column by columm, page by page, etc. Let's index into several variables in several ways:

In [16]:
println(v_float[1])
println(mat_int[3, 2])
println(mat_int[6])
println(tens_int[1,2,2])

1.0
8
8
b


We can also index into slices of a particular dimension using the ``:`` operator. If we want a particular subset of the slice we can specify the indexes ``ind1:ind2``. Julia is 1-indexed which is similiar to R but different to Python. It also doesn't do negative indexing like R and Python but it does have the special keyword ``end`` if you are unsure of the dimension length or think it might change in the future. Let's try taking some slices.

In [17]:
println(v_any[2:end])
println(mat_int[1, :])
println(tens_int[2:3, 1:3, 2])

Any[1, 1.45, 1.0 + 0.4im]
[1, 2, 3]
Any["d" "e" "f"; "h" "i" "j"]


We can also modify arrays using indexing. Simply index into the array and reasign the values. Lets try modifiying a vector and a matrix:

In [18]:
v_float[2:3] = [12, 15];
mat_int[1,:] = [60,120, 10];
println([v_float, mat_int])

Array[[1.0, 12.0, 15.0], [60 120 10; 4 5 6; 7 8 9]]


### 1.3.2 Passing by Value and Reference

This brings us to quite an important part of Julia: variable names are pointers to data. Oftentimes, we will find ourselves saying that ``var1 = var2``, for some reason or another. This is copying by *reference* not by *value* i.e. this does not make a copy of the data, it instead says ``var2`` points to the same location as ``var1``. Observe what happens when we then decide to change the first variable:

In [19]:
mat_int2 = mat_int
println(mat_int2)
mat_int[2,1] = 100000
println(mat_int2)

[60 120 10; 4 5 6; 7 8 9]
[60 120 10; 100000 5 6; 7 8 9]


Notice how ``mat_int2`` changed to reflect the change in ``mat_int``. Often we don't want that to happen and it can be a major "gotcha" bug in many peoples code. To avoid it we use the functions: ``copy`` and ``deepcopy``. ``copy`` is a shallow copy where a copy of the object is made, but any mutable fields in the object are still passed by reference. ``deepcopy`` completely dereferences the new object and is probably the most appropriate in most circumstances. Let's try it out:

In [20]:
mat_int3 = deepcopy(mat_int)
println(mat_int3)
mat_int[2,1] = 4
println(mat_int3)

[60 120 10; 100000 5 6; 7 8 9]
[60 120 10; 100000 5 6; 7 8 9]


### 1.3.2 Arrays
The ``Array`` type is denoted with the *primitive* in the first position and the *dimension* in the second position of the curly braces. It is completely general and you can use any type as the primitive. The ``Vector`` and ``Matrix`` types are actually just aliases for the ``Array`` type. 

### 1.3.3 Common Constructors 

It would be quite tedious to type out large amount of numbers and it becomes difficult to keep track of various commas, semi-colons, and spaces. It is often easier to construct arrays with some predefined functions. The most general constructor form is is ``Array{::Type, rank}(intialiser, dims...)``. Here the rank is the order of the tensor (1: vector, 2: matrix, etc...) while dims indicates the length of each dimension. The initialiser is usually left as ``undef`` which means undefined. Let's create a rank 3 tensor with dimensions (4,2,10):

In [21]:
tens = Array{Float64, 3}(undef, 4, 2, 10)

4×2×10 Array{Float64, 3}:
[:, :, 1] =
 2.23103e-314  2.23103e-314
 2.23103e-314  2.23103e-314
 2.23103e-314  2.23103e-314
 2.23103e-314  2.23103e-314

[:, :, 2] =
 2.23103e-314  2.23103e-314
 2.23103e-314  2.23103e-314
 2.23103e-314  2.2313e-314
 2.23103e-314  2.2313e-314

[:, :, 3] =
 2.2313e-314  5.0e-324
 2.2313e-314  5.0e-324
 2.2313e-314  5.0e-324
 5.0e-324     0.0

[:, :, 4] =
 0.0  0.0
 0.0  0.0
 0.0  5.0e-324
 0.0  5.0e-324

[:, :, 5] =
 2.0e-323  5.0e-324
 5.0e-324  5.0e-324
 5.0e-324  1.0e-323
 5.0e-324  5.0e-324

[:, :, 6] =
 0.0       5.0e-324
 0.0       5.0e-324
 5.0e-324  0.0
 0.0       0.0

[:, :, 7] =
 0.0  0.0
 0.0  0.0
 0.0  0.0
 0.0  0.0

[:, :, 8] =
 0.0  0.0
 0.0  0.0
 0.0  0.0
 0.0  0.0

[:, :, 9] =
 0.0  0.0
 0.0  0.0
 0.0  0.0
 0.0  0.0

[:, :, 10] =
 0.0  0.0
 0.0  0.0
 0.0  0.0
 0.0  0.0

Notice how all of the entries have been given a small number: it has just been plucked out of memory and is quite random. We should be careful when working with these undef intialisers in case it adds an instabilty. Let's now fill it in:

In [22]:
sz = size(tens)
for k = 1:sz[3]
    for j = 1:sz[2]
        for i = 1:sz[1]
            tens[i,j,k] = i^2 + j*k
        end
    end
end
tens

4×2×10 Array{Float64, 3}:
[:, :, 1] =
  2.0   3.0
  5.0   6.0
 10.0  11.0
 17.0  18.0

[:, :, 2] =
  3.0   5.0
  6.0   8.0
 11.0  13.0
 18.0  20.0

[:, :, 3] =
  4.0   7.0
  7.0  10.0
 12.0  15.0
 19.0  22.0

[:, :, 4] =
  5.0   9.0
  8.0  12.0
 13.0  17.0
 20.0  24.0

[:, :, 5] =
  6.0  11.0
  9.0  14.0
 14.0  19.0
 21.0  26.0

[:, :, 6] =
  7.0  13.0
 10.0  16.0
 15.0  21.0
 22.0  28.0

[:, :, 7] =
  8.0  15.0
 11.0  18.0
 16.0  23.0
 23.0  30.0

[:, :, 8] =
  9.0  17.0
 12.0  20.0
 17.0  25.0
 24.0  32.0

[:, :, 9] =
 10.0  19.0
 13.0  22.0
 18.0  27.0
 25.0  34.0

[:, :, 10] =
 11.0  21.0
 14.0  24.0
 19.0  29.0
 26.0  36.0

That is much easier than typing it all out! Another set of useful constructors are the ``zeros(dims...)`` and ``ones(dims...)`` functions which create a tensor of ones, or zeros.

In [23]:
v = zeros(10)

10-element Vector{Float64}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

Once an array has been created we can add to it in a number of ways. The three easiest functions are append!, prepend!, and push!. The bang (``!``) operator after a function in Julia conventionally means that the operation is performed inplace. Let's compare:

In [24]:
append!(v, 10)

11-element Vector{Float64}:
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
 10.0

In [25]:
prepend!(v, [2,124])

13-element Vector{Float64}:
   2.0
 124.0
   0.0
   0.0
   0.0
   0.0
   0.0
   0.0
   0.0
   0.0
   0.0
   0.0
  10.0

In [26]:
push!(v, 12.0)

14-element Vector{Float64}:
   2.0
 124.0
   0.0
   0.0
   0.0
   0.0
   0.0
   0.0
   0.0
   0.0
   0.0
   0.0
  10.0
  12.0

To delete an element in a list we have the function ``deleteat`` which takes an array and a list of indices (Cartesian or Linear) and removes those elements:

In [27]:
deleteat!(v, [1, 5, 6, 11])

10-element Vector{Float64}:
 124.0
   0.0
   0.0
   0.0
   0.0
   0.0
   0.0
   0.0
  10.0
  12.0

Finally, we can concatenate arrays using ``vcat`` and ``hcat``. These contactenate horizontally and vertically, repsectively. The dimensions of the array must agree. We can add abritarily add vectors into a single concatentation call. Let's try it out:

In [28]:
v1 = rand(10);
v2 = ones(10);

In [29]:
vcat(v1, v2)

20-element Vector{Float64}:
 0.3234549041569805
 0.33757724199685757
 0.8753258246633571
 0.11045430730668959
 0.9889592065094526
 0.4514603983142651
 0.32465831894636243
 0.5643708716785313
 0.629396468709196
 0.7718630939379649
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0

In [30]:
hcat(v1, v2)

10×2 Matrix{Float64}:
 0.323455  1.0
 0.337577  1.0
 0.875326  1.0
 0.110454  1.0
 0.988959  1.0
 0.45146   1.0
 0.324658  1.0
 0.564371  1.0
 0.629396  1.0
 0.771863  1.0

In [31]:
hcat(v1, v2, v1)

10×3 Matrix{Float64}:
 0.323455  1.0  0.323455
 0.337577  1.0  0.337577
 0.875326  1.0  0.875326
 0.110454  1.0  0.110454
 0.988959  1.0  0.988959
 0.45146   1.0  0.45146
 0.324658  1.0  0.324658
 0.564371  1.0  0.564371
 0.629396  1.0  0.629396
 0.771863  1.0  0.771863

In [32]:
vcat(hcat(v1, v2, v1), hcat(v2, v1, v2))

20×3 Matrix{Float64}:
 0.323455  1.0       0.323455
 0.337577  1.0       0.337577
 0.875326  1.0       0.875326
 0.110454  1.0       0.110454
 0.988959  1.0       0.988959
 0.45146   1.0       0.45146
 0.324658  1.0       0.324658
 0.564371  1.0       0.564371
 0.629396  1.0       0.629396
 0.771863  1.0       0.771863
 1.0       0.323455  1.0
 1.0       0.337577  1.0
 1.0       0.875326  1.0
 1.0       0.110454  1.0
 1.0       0.988959  1.0
 1.0       0.45146   1.0
 1.0       0.324658  1.0
 1.0       0.564371  1.0
 1.0       0.629396  1.0
 1.0       0.771863  1.0

### 1.4 Type Conversions and Hierarchy

Type conversion can be performed on arrays. Let's try and convert our Float64 vector into a Float32 vector and our Float64 Matrix into a Float16 matrix.

In [41]:
v_f32 = Array{Float32, 1}(v_float)
m_f16 = Array{Float64, 2}(mat_int)

3×3 Matrix{Float64}:
 60.0  120.0  10.0
  4.0    5.0   6.0
  7.0    8.0   9.0

That worked quite well. Notice that we used ``Array{Float32, 1}`` and ``Array{Float16, 2}`` instead of ``Vector{Float32}`` and ``Matrix{Float16}``. This is because ``Vector`` and ``Matrix`` are special aliases for Array{}. Let's create an array of arrays of dimension 1. In many languages this is how you create a matrix.

In [71]:
array_array = Array{Array{Float64,1}, 1}
println(array_array)
println(array_array <: Matrix)
println(Matrix <: array_array)
println(typeof(zeros(2,2)) <: Matrix)

Vector{Vector{Float64}}
false
false
true


Matrix{Float64} (alias for Array{Float64, 2})

Notice that Julia has constructed this new type, and correctly recognised that it is distinct (and not in the type hierarchy) from ``Array{Float64,2}``. Functions designed for matrices may not work for vectors of vectors, and we shouldn't expect them to unless they are explicitly extended to.

In [23]:
Matrix{Float64} <: Array{Array{Float64, 1}, 1}

false

## 2 Logic and Control Flow

An essential feature of *all* programming is control flow. These are structures that allow us to implement the logic of our program. An essential feature of control flow is the notion of Booleans and truth statements. A boolean is a type which indicates a logical value: ``true`` or ``false``. These are usually the outputs of logical queries which are most commonly constructed with the binary operators. Some common and useful ones are: and (&&), or (||), less than (<), greater than (>), equal to (==), and greaterequal/lessequal (>=, <=). These are not exhaustive, Julia has many others such as those from [set logic](https://docs.julialang.org/en/v1/base/collections/) or the xor operator and we can consult the documentation for further details. Let's try some logical statements:

In [24]:
a = 1
b = 2.0
c = "three"

println( a > b )
println( a <= b )
println( a == b )
println( a == c )
println( (a<b) && (b>=a) )
println( (a<b) && (b==c) )

false
true
false
false
true
false


### 2.1 Logical Negation

Logical negation is another important part of control flow. In Julia it is controlled through the ``!`` operator. The bang! precedes the logical value. Again, it follows the rules of Boolean logic and so statments can be contactenated and easily readable. Let's try negating a few logical statements. 

In [25]:
println(!true)

println(!(a > b))

println(!(a > b) && (a <= b))

println((a > b) && !(a == b))

println(!((a > b) || (a == b)))

false
true
true
false
true


### 2.2 Control Flow
Control flow in Julia is done predominately in blocks. Let's go through the ``if``, ``for``, and ``while`` control flows as these are most common. All control flow blocks end in the ``end`` keyword.

### 2.2.1 If Statements
The ``if`` block has three keywords: ``if``, ``elseif``, and ``else``. The latter are unecessary. The ``if`` and ``elseif`` keywords are followed directly by logical statements which evaluate to Boolean: ``true`` or ``false``. After each keyword there is a section of code to execute if the statement is ``true`` with the ``else`` block executing if no proceeding blocks are true. There can be an abritrary number of ``elseif`` statements. Let's construct a few ``if`` statements to see how they operate:

In [26]:
obj1 = "a string";
obj2 = 2.0;
obj3 = 30;

In [27]:
if typeof(obj1) == String
    println("It's " * obj1)
end

It's a string


In [28]:
if typeof(obj1) == Float32
    println("It's " * obj1)
else
    println("It's not " * obj1)
end

It's not a string


In [29]:
if typeof(obj1) == Float32
    println("It's " * obj1)
elseif obj2 < obj3
    println("Ordering...")
    println([obj2, obj3])
end

Ordering...
[2.0, 30.0]


In [30]:
if typeof(obj1) == Float32
    println("It's " * obj1)
elseif obj3 < obj2
    println("Ordering...")
    println([obj3, obj2])
else
    println("Ordering...")
    println([obj3, obj2])
end

Ordering...
[30.0, 2.0]


In [31]:
if typeof(obj1) == Float32
    println("It's " * obj1)
elseif obj3 < obj2
    println("Ordering...")
    println([obj3, obj2])
elseif (typeof(obj3) == Int64) | (typeof(obj2) == Int64)
    println("There is a rogue interger in here.")
else
    println("Ordering...")
    println([obj3, obj2])
end

There is a rogue interger in here.


### 2.2.2 Loops

A loop is another more complicated form of control. Arguably the most common form of loop is the ``for`` loop. The for loop in Julia is initiated with the ``for`` keyword, a loop variable, and some form of interable e.g. an index range or vector. Then, the compiler loops sequentially through the iterable and assigns it to the loop variable while executing the code inside the block. The variable assignment can be using the ``=`` operator or the ``in`` keyword. Let's try it out: 

In [93]:
println("Using the = assignment")
for i = [1,2,3,"af"]
    println(i)
end

println("\nUsing the `in` assignment")
for i in [1,2,3,"af"]
    println(i)
end

Using the = assignment
1
2
3
af

Using the `in` assignment
1
2
3
af


Let's try something a little more complicated. There is a Julia function called ``iseven`` which queries a number and checks if it is even. Let's embed an ``if`` statement inside a ``for`` loop which, for the first 15 natural numbers, prints out the square of the number if it is even and the square root of a number if it is odd.

In [33]:
for i = 1:15
    if iseven(i)
        println(i^2)
    else
        println(sqrt(i))
    end
end

1.0
4
1.7320508075688772
16
2.23606797749979
36
2.6457513110645907
64
3.0
100
3.3166247903554
144
3.605551275463989
196
3.872983346207417


The ``while`` loop is another very useful looping construct. It is typically used when the dimension of the problem is not known and it often, although not necessarily, has a more non-linear structure and less determined run time. It is formed with the ``while`` keyword and a logical statement which usually is dependent on something inside the loop. The ``while`` loop runs until the statement reads false. Let's construct a simple while loop that performs the same task as the ``for`` loop above but with the ``isodd`` function.

In [153]:
i = 15
while i > 1
    i = i - 1
    if isodd(i)
        println(i^2)
    else
        println(sqrt(i))
    end
end

3.7416573867739413
169
3.4641016151377544
121
3.1622776601683795
81
2.8284271247461903
49
2.449489742783178
25
2.0
9
1.4142135623730951
1


Notice how loop variable ``i`` gets incremented by 1 each time and is therefore guarenteed to make the logical statement false at some point. A ``while`` loop will accept any Boolean: `` while true [some code block] end`` is a valid and executable code. However, the ``true`` will never turn to ``false`` in that example and would require killing the Julia process. Infinite loops are problems in ``while`` loops because oftentimes the runtime is not known beforehand --- if we haven't carefully checked that our code will terminate we might not know if we are going to be waiting for a long time, or forever.

### 2.2.3 Increment Operators

In both of the loops we referenced ``i``  in an operation that increased or decreased it. In Julia there are a number of operators that allow you to concisely express this (you may be familiar with them from other languages). They are:
1. ``i+=val`` (increases i by val)
2. ``i-=val`` (decreases i by val)
3. ``i*=val`` (multiplies i by val)
4. ``i/=val`` (divides i by val)

### 2.2.4 Syntatic Sugar

There are two forms of syntatic sugar that many people like to employ in Julia. The first is the ternary operator which lets us write ``if`` statements in a single line, and the other is loop comprehension which lets us write ``for`` statements in a single line. In general, these decrease the readability of our code so they can be avoided. They can be very helpful in anonymous functions calls. Also, other people very much like using them, so watch out!

The ternary operator takes the form: ``(Boole ? Statement1 : Statement2)`` where Boole indicates the logical query, followed by ?, then if this is true Statement1 is executed, the : indicates else and Statement2 is executed if the logical query is false.

The list comperhension takes the form: ``[f(i) for i in iter]`` where i is the loop variable, iter is the collection being iterated over, and f(i) is the loop execution block. The result is a vector containing the sequential applications of ``f`` to the loop variable.

Let's try these two statements out:

In [35]:
(true == false ? println("That's true") : println("That's false"))

That's false


In [36]:
(true == (!false) ? println("That's true.") : println("Unless...")) 

That's true.


In [37]:
vec_even_squares = [i^2 for i in 0:2:10]

6-element Vector{Int64}:
   0
   4
  16
  36
  64
 100

## 3.0 Functions

We have actually already covered many functions up until this point: some using traditional function brackets notation and others being binary operators. Fortunately, they were all pretty inituitive in how they worked! Functions are much more general than these though and Julia has first class support for functions! 


### 3.1 Inline Functions
The first function type we will cover is inline functions. These look like the functions you probably first worked with in high school - they are defined in one line. Let's define a simple function: $ f(x) = x^2 $:

In [38]:
f(x) = x^2

f (generic function with 1 method)

In [39]:
println([f(3), f(2.0), f(1 + 2im), f("BAA"), f(true), f(false)])

Any[9, 4.0, -3 + 4im, "BAABAA", true, false]


That worked pretty well - it even seemed to generalise to any data type! What if we wanted to just define a function that worked only on a specific data type. In Julia this is done using the ::Type operator in the function arguments. Let's define a square root function that only works with Float32 data:

In [40]:
root32(x::Float32) = sqrt(x)

root32 (generic function with 1 method)

In [41]:
root32(4)

LoadError: MethodError: no method matching root32(::Int64)
[0mClosest candidates are:
[0m  root32([91m::Float32[39m) at In[40]:1

The first thing we notice is that it no longer seems to "just work". The reason that the first function worked on so many data types is because we didn't specify what data type it ought to be so Julia infers at as the Any type and the function will accept any input. The base square operation happened to be defined for all the data we through at it. The ``root32`` function *needs* Float32 data:

In [42]:
root32(4f0)

2.0f0

That's much better - it's working now. The function might still have errors but provided we give it correctly type data Julia will be able to dispatch this method. For example, if we take the square root of a negative number we should get an error, but it wont be a *method* error. Let's try taking the root of a negative number:

In [43]:
root32(-4f0)

LoadError: DomainError with -4.0:
sqrt will only return a complex result if called with a complex argument. Try sqrt(Complex(x)).

### 3.2 Multiple Dispatch (Advanced)

This brings us to one of the key selling features of Julia: multiple dispatch. Multiple dispatch essentially allows us to overload a function definition so that it can apply (dispatch) different methods based on the data type it receives. This is incredibly important: many functions we think of as quite intuitive have many different specific meanings and implementations based on a host of factors. Think about squaring a number and squaring a matrix; they both "make sense" and the operator should remain the same, but the computational procedures are very different. Multiple dispatch allows us to keep conventions that make sense, decide on faster routines if the high precision is not required, or different more precise methods if it is. Many of Julia's base operators are already overloaded.

Suppose that we want to extend our function to accept Complex32 arguments. We might remember that the square root of a complex number has two values. We would like to keep things simple so we will just extend the definition to produce that:

In [44]:
root32(x::ComplexF32) = (sqrt(x.re^2 + x.im^2) + x.re) / 2 + x.im/abs(x.im) * (sqrt(x.re^2 + x.im^2) - x.re) / 2

root32 (generic function with 2 methods)

Notice now that our ``root32`` function has two methods. Let's try it out:

In [45]:
root32(1f0 + 3f0im)

3.1622777f0

In [46]:
root32(1f0 + 0f0im)

NaN32

That's interesting: we gave it a complex number with a zero value imaginary part and it returned a Not a Number value. Why? Surely it should just give the root of 1 which is well defined? The problem is that because we gave it a complex number it dispatched to the method defined for complex numbers which has a division by the imaginary component. Since our imaginary component is zero we get a division-by-zero error. To fix this we can overload a special case into the function:

In [56]:
root32(x::ComplexF32) = (x.im != 0 ? root32(x) : root32(Float32(x.re)))
root32(1f0 + 0f0im)

1.0f0

Now that works properly. Why bother with all of this? First, it highlights how difficult it can be to write code with multiple types. The best practice here is to start with AbstractAny types and let the compiler do its own type inference and once the code is working to progressively make the functions more restrictive. The second is performance. As we have seen the compiler is very flexible and can mutate types and perform type conversions to make certain operations make sense: these come at a cost. By specifiying all of our types at runtime we let the compiler know what to expect and by making our code type-stable we can get substantial performance gains. Type mutations add a lot of bloat.

### 3.3 Optional and Keyword Arguments

We might find ourselves wanting to give additional arguments to our function that function more like parameters. A natural example might be coding the funtion $ f(x; a) = a \cdot x^2 $. Suppose we normally expect $a = 1$ but want to code it some flexibilty for when it isn't. To do this we use an optional argument by putting an ``=`` operator in the function definition:

In [58]:
fa_opt(x, a=1) = a * x^2

fa_opt (generic function with 2 methods)

In [62]:
fa_opt(2)

4

In [64]:
fa_opt(2, 3)

12

This can be immensely useful especially for edge cases. However, when we start writing functions with many parameters it can be difficult to remember the order they are in. The keyword argument methodology allows you to reference arguments by symbol rather than location which offers much more convenience. Keyword arguments are specified after a semi-colon in the function definition and must be called with their symbol:

In [65]:
fa_key(x; a=1) = a * x^2

fa_key (generic function with 1 method)

In [66]:
fa_key(2)

4

In [68]:
fa_key(2; a=2)

8

In [70]:
fa_key(2; 2)

LoadError: syntax: invalid keyword argument syntax "2" around In[70]:1

### 3.5 Generic Functions

So far we have focused on in-line functions because they are readable and feel more intuitive to functions we have seen before. There is only so much we can do on one line, however, and we need a function structure that allows us to include multiple statements and a generic block to execute code. The ``function`` code block provides just that. Like the control flows it also requires an ``end`` statement and it has another optional component ``return``. Whenever the ``return`` keyword is found the function will break out and return whatever value is in the return statement. If there is no return it will return the last line it gets to. All of the multiple dispatch, optional arguments, and keyword arguments apply to generic functions. Let's write our $f(x; a) = a \cdot x^2 $ function in this format:

In [72]:
function F(x; a=1)
    res = a * x^2
    return res
end

F (generic function with 1 method)

For that type of function it is probably more readable and far less typing to use an inline function. However, let's consider a more complicated squaring function which uses control flow to generalise the squaring behaviour to multiple different array types. We will use the ``size`` function to determine the dimensions of the array.

In [89]:
function asquare(x; a=1)
    sz = size(x)
    if length(sz) == 0 
        # this is a number
        return a * x^2
    elseif length(sz) == 1
        # this is a vector
        tmp = 0
        for i = 1:sz[1]
            tmp += x[i]^2
        end
    elseif length(sz) == 2 && (sz[1] == sz[2])
        # this is a square matrix
        tmp = zeros(sz)
        for j = 1:sz[2]
            for i = 1:sz[1]
                for k = 1:sz[1]
                    tmp[i,j] = a * x[i,k] * x[k,j]
                end
            end
        end
    else
        println("There is a problem")
        return nothing
    end
    
    return [x, tmp]
end

asquare (generic function with 1 method)

In [90]:
x = [1 2 ; 3 4]
asquare(x; a=pi)

2-element Vector{Matrix{Float64}}:
 [1.0 2.0; 3.0 4.0]
 [18.84955592153876 25.132741228718345; 37.69911184307752 50.26548245743669]

That was quite a ride! The function seems to works well - you can try it out for different values and types for ``x``. It is quite a complicated function which would be very difficult to do in a single line. It might be possible to achieve with multiple dispatch but it would be difficult to keep track of. In general, the ``function`` blocks provide the best space in which to work.

### 3.6 Parsing by reference/value and the bang operator

We have already discussed parsing by reference or by value in the context of variable assignment. These principles extend to functions.

In [3]:
a = Any[1 2 3; 4 5 6]
function f(a)
    a[3] = "A"
end
display(a)
f(a)
display(a)

2×3 Matrix{Any}:
 1  2  3
 4  5  6

2×3 Matrix{Any}:
 1   "A"  3
 4  5     6

When we see a bang (!) asscociated with a function we usually can take it to mean that function is *in-place* and non-memory allocating. Either, it will allocate the memory to the object it is provided, or a data vector is parsed along with it and it makes the assignments there. Either way it doesn't allocate. Some functions have both options and these can have uses for different cases. Consider the ``sort`` and ``sort!`` functions:

In [159]:
println("This is the vector to be sorted.")
vec = rand(5)
println(vec)

println("\nThis creates a new vector and doesn't change the old vector:")
sorted_vec = sort(vec)
println(sorted_vec)
println(vec)

println("\nThis changes the vector into a sorted vector:")
sort!(vec)
println(vec)

This is the vector to be sorted.
[0.9578460706360399, 0.47717545214785917, 0.3650891468156212, 0.3608949175677356, 0.7236792039609115]

This creates a new vector and doesn't change the old vector:
[0.3608949175677356, 0.3650891468156212, 0.47717545214785917, 0.7236792039609115, 0.9578460706360399]
[0.9578460706360399, 0.47717545214785917, 0.3650891468156212, 0.3608949175677356, 0.7236792039609115]

This changes the vector into a sorted vector:
[0.3608949175677356, 0.3650891468156212, 0.47717545214785917, 0.7236792039609115, 0.9578460706360399]


### 3.7 Anonymous Functions

Functions are first class objects in julia. They can be assigned to variable names (as we have been doing) or they can be assigned anonymously (without names). These are known as lamda expressions in R and Python. There are two ways to write a lambda expression:

In [91]:
x -> x^2 + 1

#12 (generic function with 1 method)

In [92]:
function(x)
    return x^2 + 1
end

#14 (generic function with 1 method)

Both of these give a generic function method and a number instead of a name. The number is for the compiler. Why bother with anonymous functions? The major use case is for them to be used as arguments for other functions. They are very useful in the functional programming paradigm. We will cover some of these special functions in a later workbook.

## 3.8 Packages

Writing our own functions and macros can be rewarding: it is a very good way to learn and understand how an algorithm works. However, it is very time consuming and often our first attempts will not be as well optimised or perhaps as organised as others who have come before us. In order to not reinvent the wheel we turn to Packages which are folders of code written by others that we can download and run.

To import a package and all its functions into the namespace we simply run the code ``using Package``, where ``Package`` is the package name. Then, all the functions exported by the package are in the name space and if there are no naming conflicts we can call them directly. If multiple names exist we can always fall back on the notation ``Package.func`` where ``func`` is the function name for the package.

The package system in Julia is excellent! To access it from the REPL just hit ``]``. This brings you into the package manager. To exit just hit ``backspace``. Inside the package manager we can ``add Package``, ``remove Package``, and ``build Package``, where ``Package`` is the package name. We can also do this through the Package package called Pkg. Since we are in a notebook we will use the ``Pkg.add()`` function to add the packages ``StatsBase``.

In [124]:
using Pkg
Pkg.add("StatsBase")

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`


In [123]:
using StatsBase
rand_vec = rand(100)
println(mean(rand_vec))

0.5863310687041419


The ``StatsBase`` package exports another useful function ``std`` to compute the standard deviation of a series of numbers. Suppose that we had also written a (fairly useless) function which **s**quared **t**wo **d**oubled **x** and also called it ``std(x)``.

In [134]:
function std(x)
    return (2^2, 2 * x)
end

std (generic function with 1 method)

What happens if we call the ``std`` function on our random vector?    

In [135]:
std(rand_vec)

(4, [1.3903144829116534, 1.6167085700586565, 1.1774375904207675, 0.6751688918060605, 0.5785784074588096, 0.7851893768856046, 1.4480466462390202, 1.6502250399801064, 1.1948370951141465, 1.695519147407488  …  0.859159593261003, 0.8124285429014788, 1.870460218092279, 1.7089009329908997, 1.5621564518310973, 1.0143041685519565, 1.7375532649773386, 1.6308744745865282, 0.30251421567007064, 1.5557770054808162])

This is not exactly what we wanted, and fortunately we can identify that we haven't got a standard deviation fairly quickly. The naming conflict means that the compiler has chosen the highest level definition for the function it can find. Let's insist that is use the ``StatsBase`` method.

In [136]:
StatsBase.std(rand_vec)

0.27009659495816657

## 3.9 Macros

Macros are useful sets of instructions that perform multiple tasks and general provide extensible functionality to the language. Julia employs LISP like macros that are denoted with the ``@`` character. Macros are often provided to time operations, profile operations, sync operations, and provide peformance enhancements. A useful list of macros are:

1. @time 
2. @elapsed (allows us to save the time into a variable)
3. @btime (benchmark times code: BenchmarkTools package)
4. @benchmark (benchmarks code and gives summary: Benchmark Tools package)
5. @profile (profiles code: Profile package)
6. @sync (synchronises threads: Distributed package)
7. CUDA.@sync (synchronises cuda thread: CUDA package)
8. @inbounds (turns off bounds checking)
9. @simd (activitates SIMD operations)
10. @. (broadcasts)

## 3.10 Broadcasting

Most scientific languages have a notion of broadcasting: taking a function and applying it element-wise to a list. Usually, these functions are overloaded in some special way to handle both lists and numbers and one has to remember how to use these functions and which ones offer the special functionality. In Julia broacasting is easy - it's availabe to *every* function. To broadcast a function call simply add a ``.`` after its name. Let's broadcast the ``sin`` and ``exp`` functions:

In [145]:
v = collect(0:0.1:2)
broad_exp = exp.(v)
broad_sin = sin.(v)

println(broad_exp)
println()
println(broad_sin)

[1.0, 1.1051709180756477, 1.2214027581601699, 1.3498588075760032, 1.4918246976412703, 1.6487212707001282, 1.8221188003905089, 2.0137527074704766, 2.225540928492468, 2.45960311115695, 2.718281828459045, 3.0041660239464334, 3.3201169227365472, 3.6692966676192444, 4.0551999668446745, 4.4816890703380645, 4.953032424395115, 5.4739473917272, 6.0496474644129465, 6.6858944422792685, 7.38905609893065]

[0.0, 0.09983341664682815, 0.19866933079506122, 0.29552020666133955, 0.3894183423086505, 0.479425538604203, 0.5646424733950354, 0.644217687237691, 0.7173560908995228, 0.7833269096274834, 0.8414709848078965, 0.8912073600614354, 0.9320390859672263, 0.963558185417193, 0.9854497299884601, 0.9974949866040544, 0.9995736030415051, 0.9916648104524686, 0.9738476308781951, 0.9463000876874145, 0.9092974268256817]


These functions might be expected to have a broadcast call natively inbuilt. Let's try it out on our complicated function that we defined earlier: ``asquare``:

In [149]:
v_list = [1.0, 2.0, [1 2; 3 4], 4, NaN, [12 12 12; 14 14 14]]
asquare.(v_list)

There is a problem


6-element Vector{Any}:
   1.0
   4.0
    [[1.0 2.0; 3.0 4.0], [6.0 8.0; 12.0 16.0]]
  16
 NaN
    nothing

It works just as expected! This is an incredibly powerful tool to have at our disposal and it also allows us to clear our headspace about what is and isn't allowed when we write code. This allows for our code to be both more creative and expressive.
    The ``@.`` macro is a simple macro which casts the broadcast operation to every function in the code block. It is quite powerful but there aren't many times when we get a significant gain in expressivity or readability and you often find people tend to favour the ``func.`` methods.

## 3.11 Vectorisation

In other scientific languages such as Python, R, and Matlab vectorisation is taught as best practice. We often find ourselves writing a lot of pencil-on-paper linear algebra to find a way to express our code in a vectorised format. We are also taught to avoid loops like the plague. This is because the vectorised functions have been compiled in a special way to use tricks to get a speed up. Loop unrolling is often slow and comes with a performance penalty. We will make use of the ``@btime`` macro from the ``BenchmarkTools`` package to show that there is very little difference between the two in Julia:

In [120]:
using BenchmarkTools
vec_a = rand(100)
mat_a = rand(100, 100)

function matmul_test(mat, vec)
    tmp = zeros(size(mat)[1])
    for j = 1:size(mat)[1]
        for i = 1:size(mat)[2]
            tmp[i] += vec[j] * mat[i, j]
        end
    end
    return tmp
end

# check they give the same result to within machine precision
println(sum(mat_a * vec_a - matmul_test(mat_a, vec_a)) < 1e-7)

@btime mat_a * vec_a;
@btime matmul_test(mat_a, vec_a);

true
  3.245 μs (1 allocation: 896 bytes)
  4.539 μs (1 allocation: 896 bytes)


The slight edge goes to the vectorised multiplication *but* this was just rough-and-ready code with no performance enhancements. In the next notebook we will go through how to improve this code with some optimisations and we will find it actually *outperforms* the vectorised code. This is, in some sense, remarkable: it allows us to change our workflow and expand our horizons to generically complicated functions *without* the performance hit. As an example think about indexing on every even element on the vector and every odd row on the matrix and performing the same calculation: a pseudo matrix multiply. This would be difficult to vectorise and would require many more memory allocations. It's trivial in the loop format.

## 4 Functional Programming

Functional programming is a computational paradigm whereby functions are given first-class status and they are used to operate on data. This is different from the traditional object oriented paradigm whereby objects are created and are given methods (functions) and attributes (data) in their fields. Both of these paradigms are useful and very few languages (or programmers) operate in strictly one or the other; projects exist conceptually and literally as a combination of both. Scientific programming lends itself well to thinking it terms of functional programming and Julia (while it can do both) is well suited to accomodate it. 

We will go through some functions that are commonly asscociated with functional programming (but really are just very useful functions so use them how you please). They are included here because they encourage us to think about functional patterns and can improve code readability, stability, and portability! They can also make you write readable code in one line!

### 4.1 Map

The ``map(func, iter)`` function acts by taking a function and an iterable and returns a vector where the function is applied to the iterable. Try it out by finding the square of every odd number up until 100:

In [3]:
results_vec = map(x -> x^2, 1:2:100)

50-element Vector{Int64}:
    1
    9
   25
   49
   81
  121
  169
  225
  289
  361
  441
  529
  625
    ⋮
 5929
 6241
 6561
 6889
 7225
 7569
 7921
 8281
 8649
 9025
 9409
 9801

That was much more succinct than writing a ``for`` loop. Map can also take more complicated functions. Let's define a random sampling function that takes the mean of a random sample of ``n`` points:

In [7]:
function randsamp(n)
    vec = rand(n)
    return sum(vec) / length(vec)
end

randsamp (generic function with 1 method)

In [8]:
map(randsamp, 1:2:100)

50-element Vector{Float64}:
 0.7904922172298462
 0.482551143649089
 0.6373840127063914
 0.6940599254528319
 0.46294612372048455
 0.376865484980863
 0.7087015137591706
 0.6391181126049456
 0.6007631324219589
 0.48010078604877743
 0.5613503901939864
 0.4604722085752048
 0.4512239823631722
 ⋮
 0.4790393895936902
 0.5577560371636615
 0.565817267916698
 0.5310562747816733
 0.5264320932272764
 0.457929471410226
 0.4880304019025163
 0.5285810935835803
 0.5063193427165149
 0.4257356191196481
 0.49745734779945994
 0.47409289517268327

The function we provide to map doesn't have to return a single value! We can return an arbitrary type (including nothing) and get a list in return.

In [10]:
map(n -> (randsamp(n), sqrt(n)), 0:2:100)

51-element Vector{Tuple{Float64, Float64}}:
 (NaN, 0.0)
 (0.7269068806252141, 1.4142135623730951)
 (0.42519256608426903, 2.0)
 (0.5807535246637129, 2.449489742783178)
 (0.45769313406295903, 2.8284271247461903)
 (0.5245438867876088, 3.1622776601683795)
 (0.5769966039860189, 3.4641016151377544)
 (0.4285507028628169, 3.7416573867739413)
 (0.6214432215102136, 4.0)
 (0.5299172033182917, 4.242640687119285)
 (0.4449460376641139, 4.47213595499958)
 (0.5276214372987241, 4.69041575982343)
 (0.5447342562330678, 4.898979485566356)
 ⋮
 (0.5255553673891059, 8.831760866327848)
 (0.5514771500413185, 8.94427190999916)
 (0.5289544924034923, 9.055385138137417)
 (0.5634355331271025, 9.16515138991168)
 (0.48551319564334455, 9.273618495495704)
 (0.4530344007105737, 9.38083151964686)
 (0.5531464193687629, 9.486832980505138)
 (0.48374693766966365, 9.591663046625438)
 (0.48238643876251075, 9.695359714832659)
 (0.4568679233771622, 9.797958971132712)
 (0.5333708922506192, 9.899494936611665)
 (0.508187914235076, 

Similarly, we don't have to limit ourselves to just one iterable! We can provide several iterables which will be parsed as function variables in the list order that they appear in:

In [11]:
map((x,y) -> x^2 .+ y, 1:2:100, 0:2:98)

50-element Vector{Int64}:
    1
   11
   29
   55
   89
  131
  181
  239
  305
  379
  461
  551
  649
    ⋮
 6005
 6319
 6641
 6971
 7309
 7655
 8009
 8371
 8741
 9119
 9505
 9899

### 4.2 Reduce

Reduce is another useful function that acts on iterables. It requires a binary operator (think: +, \*) and it applies this operator to elements in the iterable successively accumulating the value. An easy way to write a factorial function is using the ``reduce`` function with the multiply binary operator:

In [22]:
res = reduce(*, 1:20)

2432902008176640000

In [24]:
res == factorial(20)

true

In [28]:
@btime factorial(20);
@btime reduce(*, 1:20);

  2.124 ns (0 allocations: 0 bytes)
  11.974 ns (0 allocations: 0 bytes)


That's pretty good! We can also dispatch our custom binary operations through the use of function calls. Let's multiply reduce the square of every number up to 5:

In [38]:
reduce((x,y) -> x^2*y^2,  1:5)

2751882854400

#### 4.2.1 Sum
Reducing with the + operator is the most common form of reduction: this is an array sum. Julia offers sum useful features in this regard. We can reduce along a specific dimension and we can parse a function to apply to the array  elements while reducing. Consider the following example:

In [2]:
a = rand(4,5,60)
sum(x -> 3*x, a, dims=3)

4×5×1 Array{Float64, 3}:
[:, :, 1] =
 100.665   93.1879   91.0656  78.8939  94.3963
  79.7941  82.9222   94.3989  90.4154  89.2226
  92.8048  89.9142  105.324   98.9607  88.5256
  87.0613  84.3679   83.9818  90.3253  81.0025

### 4.3 Filtering

A filter is a functional method of applying a logical test to an array and removing the elements that return false on this test. The ``filter`` and ``filter!`` functions provide this functionality through allocating a new filtered array, and modifying the array being filtered inplace. These functions accept two argument arguments: the filter (function), and the array:

In [24]:
a = rand(40) .- 0.2

40-element Vector{Float64}:
  0.7679629176826233
  0.38470814479269305
  0.11956856393909115
  0.41448194984582315
 -0.037535068265159344
  0.3658540763984542
  0.4738359747663094
  0.5884404632976896
  0.6909292447900113
  0.6856866428895949
  0.7330455538219556
 -0.013215022712058155
  0.7425996241505031
  ⋮
  0.2131187638831627
  0.502469457846114
  0.4104059380053923
  0.7222445363453327
  0.1160861014386802
  0.23602213998697347
  0.4536117090245316
  0.31603111821032465
  0.007594560965564712
  0.5408904402727459
  0.16592581545414392
  0.30646777964952293

In [26]:
b = filter(x -> x<0, a)

5-element Vector{Float64}:
 -0.037535068265159344
 -0.013215022712058155
 -0.18592684628468198
 -0.07971488527471698
 -0.031316276270590915

In [27]:
any(a .> 0)

true

In [28]:
filter!(x -> x<0, a)

5-element Vector{Float64}:
 -0.037535068265159344
 -0.013215022712058155
 -0.18592684628468198
 -0.07971488527471698
 -0.031316276270590915

In [29]:
any(a .> 0)

false

### 4.4 Finding

A special operation that we often perform on arrays is finding. The ``findall`` function allows us to query an array for values that match a logical condition. Most often this condition is provided in the form of an anonmyous function. The ``findall`` function returns a list of CartesianIndexes (or linear indexes in the case of a vector):

In [10]:
A = [1 2 4 5 0 100 -123 -12; 3 3 4 50 2 10 -3 -2]
findall(x->iseven(x), A)

10-element Vector{CartesianIndex{2}}:
 CartesianIndex(1, 2)
 CartesianIndex(1, 3)
 CartesianIndex(2, 3)
 CartesianIndex(2, 4)
 CartesianIndex(1, 5)
 CartesianIndex(2, 5)
 CartesianIndex(1, 6)
 CartesianIndex(2, 6)
 CartesianIndex(1, 8)
 CartesianIndex(2, 8)

In [9]:
findall(x -> x>0.5, rand(20))

11-element Vector{Int64}:
  1
  2
  4
  5
  9
 10
 11
 12
 16
 19
 20

The ``findfirst`` function operates in precisely the same fashion, but returns the first index that satisifies the condition:

In [12]:
findfirst(iseven, A)

CartesianIndex(1, 2)

#### 4.4.1 Finding Extrema
An extremely common index that we want to find is that of an extrema: a minimum or a maximum. Julia provides the ``findmin`` and ``findmax`` functions to this end. These return both the value of the extrema and the index it was found at. The ``maximum`` and ``minimum`` functions give just the extrema value, while the ``argmax`` and ``argmin`` functions give just the index:

In [13]:
findmin(A)

(-123, CartesianIndex(1, 7))

In [14]:
minimum(A)

-123

In [15]:
argmin(A)

CartesianIndex(1, 7)

In [16]:
A[argmax(A)] == maximum(A)

true

If there are multiple extrema these functions will report the first instance where the extrema is found:

In [23]:
v = vcat(ones(5), -ones(5))

10-element Vector{Float64}:
  1.0
  1.0
  1.0
  1.0
  1.0
 -1.0
 -1.0
 -1.0
 -1.0
 -1.0

In [21]:
argmax(v)

1

In [22]:
findmin(v)

(-1.0, 6)

## 5.0 Basic Worflow: Notebooks, REPL, Scripting,  Some Useful Funcitons
Julia has a number of different ways that we can interact with it. We have been using a notebook and should be fairly familiar with by now. The most common method is by using the REPL which is a shell like environment which can be accessed by entering ``julia`` into the terminal. In the REPL we can enter commands and get results. Try it out now.

Sometimes we want to repeatedly rerun a series of commands. For this we type them into a script with file extension .jl. We can include the script with the ``include("path/to/script.jl")`` command which is similiar to R's ``source`` command. We can also execute them directly from the terminal with ``julia path/to/script.jl``. In general, it is not advisable to use the latter method because it starts up a new process every time and all functions need to be precompiled again. This makes it very slow for scripting work. The common practice is to open a single Julia session and run all scripts from inside the REPL. We will cover a slightly more advanced workflow in the next notebook.

### Useful Functions
Here is a (non-comprehensive) list of useful functions. We will found ourselves coming back to these again, and frequently. This is just to get you started and I would encourage that you use this as a springboard while adding your own entries for often used methods. I am sure that I will have forgotten, or neglected, some very valuable functions!


1. Base
    - +,-,/,\*,^
    - exp, sin, cos, tan
    - factorial
    - . (broadcast)
    - sort(tensor, dims=(1,2...), reverse=Boole)
    - size
    - append, prepend, append!, preprend!, push!
    - ones(dims...), zeros(dims...)
    - map, reduce, filter, filter!
    - rand(), rand(n), rand([x0, x1], n)
    - sum(x), sum(x, dims=(tuple...)), sum(lambda, x, dims...)
    - @time, @elasped
    - &, |, &&, ||, !
    - any, all
    - iseven, isodd, isnan
    - maximum, minimum, argmax, argmin, findmax, findmin
    - vcat, hcat
2. Random:
    - rand, randn, randexp
    - randperm
    - Random.seed!(seed_number)
3. StatsBase
    - mean
    - var
    - std
    - sample
4. BenchmarkTools and Profile
    - @benchmark
    - @profile
    - Profile.print()
    - @btime, @belapsed
4. Distibruted and SharedArrays
    - @distributed
    - @sync
    - SharedArray
    - pmap
5. Plots
    - plot
    - plot series types: :scatter, :line, :histogram, :contour, :contourf, :plot3d, :bar, :histogram2d
    - @layout
    - savefig
6. DataFrames, JLD, CSV/Delimited Files
    - DataFrame
    - save / load
    - writecsv, readcsv, CSV.File()
    - writedlm, readdlm
7. LinearAlgebra
    - tr, span
    - eigen, eigenvalues
    - det
    - factorise
    - dot